# PYTORCH SAMPLES

Scripts derived from PyTorch Tutorial,  mainly Object Classification @ MNIST.

#### [Story](https://www.pivotaltracker.com/story/show/188139132)

#### [Source](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#### Load Data

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [3]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [5]:
for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


#### Create Models

In [6]:
# Get devide: cuda, mps or `cpu` on availability
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Operational device is {device.upper()}")

Operational device is CPU


In [7]:
# Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### Optimize Model Parameters

#### Loss function and Optmizer to train a model

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
# loop to make prediciotns on training dataset and backpropagate prediction error to adjust model's paramas

def train (dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # predicion Error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
# check the model performance against test dataset to ensure that is learning

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device)
            pred = model(X)
            test_loss =+ loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f}\n")


In [12]:
# itereations epochs
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
---------------------
loss: 2.297179 [   64/60000]
loss: 2.289219 [ 6464/60000]
loss: 2.266188 [12864/60000]
loss: 2.269588 [19264/60000]
loss: 2.251293 [25664/60000]
loss: 2.211892 [32064/60000]
loss: 2.226571 [38464/60000]
loss: 2.183880 [44864/60000]
loss: 2.184382 [51264/60000]
loss: 2.151525 [57664/60000]
Epoch 2
---------------------
loss: 2.153297 [   64/60000]
loss: 2.145521 [ 6464/60000]
loss: 2.078375 [12864/60000]
loss: 2.102457 [19264/60000]
loss: 2.057517 [25664/60000]
loss: 1.985422 [32064/60000]
loss: 2.015377 [38464/60000]
loss: 1.922487 [44864/60000]
loss: 1.933521 [51264/60000]
loss: 1.868200 [57664/60000]
Epoch 3
---------------------
loss: 1.888111 [   64/60000]
loss: 1.863826 [ 6464/60000]
loss: 1.729683 [12864/60000]
loss: 1.780755 [19264/60000]
loss: 1.689441 [25664/60000]
loss: 1.625808 [32064/60000]
loss: 1.649539 [38464/60000]
loss: 1.538236 [44864/60000]
loss: 1.576300 [51264/60000]
loss: 1.481310 [57664/60000]
Epoch 4
---------------------
loss: 1.55

#### Save Model

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


#### Loading Models

In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [16]:
# Make predictions with the model

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
